In [1]:
import pandas as pd
import numpy as np
import pyautogui as gui
import pyperclip
from datetime import datetime

In [2]:
gui.size()

Size(width=1920, height=1080)

## Setup

In [11]:
def getURLData(filename='Contest Reference.xlsx', sheetname='Scraping'):
    """
    Get data from Excel. Basis to rewrite data back to Excel.
    """
    return pd.read_excel(filename, sheet_name=sheetname)


def parseIDs(url):
    """
    Parse Game ID and Contest ID from URL string. 
    Returns a tuple.
    """
    ind = url.find('contests/')
    return url[ind + 9 : ind + 14], url[ind + 15 : ind + 24]


def openURL(url):
    """
    Opens URL in browser. Assumes browser is open and maximized.
    
    Must be signed in to FanDuel to load page properly.
    """
    # click into search bar
    gui.click(200, 60)
    
    # paste and enter
    gui.typewrite(url)
    gui.hotkey('enter', interval=.2)
    
    
def repositionFanDuel():
    """
    Repositions webpage for all contestants to be in view
    by scrolling down twice.
    """
    gui.click(950, 1020, clicks=2)
    

def locateKeys():
    """
    Returns Boxes for 'Next >' button and 'Position' header.
    Sensitive to pixels being identical to originals.
    
    Raises error if both are not found. 
    """
    FIRST = gui.locateOnScreen('images/first.png')
    NEXT = gui.locateOnScreen('images/next.png')
    POSITION = gui.locateOnScreen('images/position.png')
    
    if not FIRST:
        # raise ValueError('FIRST not found on screen.')
        print('FIRST not found on sreen. Using default value.')
        FIRST = (666, 333)
    if not NEXT:
        # raise ValueError('NEXT not found on screen.')
        print('NEXT not found on screen. Using default value.')
        NEXT = (888, 333)
    if not POSITION:
        # raise ValueError('POSITION not found on screen.')
        print('POSITION not found on screen. Using default value.')
        POSITION = (40, 405)
        
    return FIRST, NEXT, POSITION

## Scraping

In [12]:
def scrape(pages=10, min_patience=.5, POSITION=(0,0), NEXT=(0,0)):
    """
    Scrape raw data while iterating through contestants on each page.
    Patience is the minimum between contestants. If copy results in same
    string as the last copy, it will wait and try again.
    
    Assumes page has been set up.
    """
    time_start = datetime.now()

    start = POSITION[1] + 36
    delta = 36
    PATIENCE = min_patience
    data = []
    old_text = ''

    # iterate through page
    for j in range(pages):
        
        # iterate through 'position'
        for i in range(10):
            # select the position, wait to load
            gui.click(x=POSITION[0], y=(start + i * delta), interval=PATIENCE)

            try:
                if new_text != '':
                    pass
            except:
                new_text = ''
            while new_text == old_text:
                # copy all
                gui.hotkey('ctrl', 'a', interval=.1)
                gui.hotkey('ctrl', 'c', interval=.1)

                new_text = pyperclip.paste()

            # append clipboard to data
            data.append(new_text)
            
            # reset old text
            old_text = new_text

        gui.click(NEXT[0] + 5, NEXT[1] + 5, interval=max(1, PATIENCE*3))

    time_stop = datetime.now()
    print(time_stop - time_start)
    
    return data

## Turning the data into information

In [5]:
def cleanRawData(string):
    """
    Convert string to Series with meaningful information.
    """
    ser = pd.Series(string.replace('\n', '').split('\r'))
    
    # use the index of the last 'WON' to start
    start = ser[ser == 'WON'].index[-1] - 5
    end = ser[ser == 'ABOUT'].index[0]
    ser = ser.iloc[start : end]
    
    # indices of roster info
    info = np.array([0, 1, 2, 4, 6]) + start
    
    # if player has not set lineup
    if ser.str.contains("Your opponent hasn't set a lineup").any():
        temp = pd.Series([
            ser.iloc[0], # exp
            ser.iloc[1], # username
            ser.iloc[2], # position
            ser.iloc[4].replace('$', ''), # winnings
            0, # score
            np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, # players
            0, 0, 0, 0, 0, 0, 0, 0, 0 # scores
        ])
        
        if temp.iloc[0].find('user') < 0:
            temp.iloc[0] = 'Noob'
        
        return temp
        
    
    # indices of positions
    positions = np.array([
        ser[ser == 'QB'].index[0],
        ser[ser == 'RB'].index[0],
        ser[ser == 'RB'].index[1],
        ser[ser == 'WR'].index[0],
        ser[ser == 'WR'].index[1],
        ser[ser == 'WR'].index[2],
        ser[ser == 'TE'].index[0],
        ser[ser == 'FLEX'].index[0],
        ser[ser == 'DEF'].index[0],
    ])
    
    # all indices include info, names, and individual points
    indices = np.concatenate([info, positions + 1, positions + 2])
 
    temp = ser.loc[indices]
    
    # clean experience if needed
    if temp.iloc[0].find('user') < 0:
        temp.iloc[0] = 'Noob'
        
    # clean dollar sign
    temp[start + 4] = temp[start + 4].replace('$', '')
        
    return temp


def cleanAllData(data):
    """
    Clean all contestants' strings and create DataFrame.
    """
    data_dict = {}
    count = 1
    
    for e in data:
        temp = cleanRawData(e)
        
        # reindex
        temp.index = ['Experience', 'Username', 'Position', 'Winnings', 'Score',
                      'QB', 'RB1', 'RB2', 'WR1', 'WR2', 'WR3', 'TE', 'FLEX', 'DEF',
                      'P_QB', 'P_RB1', 'P_RB2', 'P_WR1', 'P_WR2', 'P_WR3', 
                      'P_TE', 'P_FLEX', 'P_DEF']
              
        # convert numerics
        numeric_ind = ['Winnings', 'Score', 'P_QB', 'P_RB1', 'P_RB2', 'P_WR1', 
                       'P_WR2', 'P_WR3', 'P_TE', 'P_FLEX', 'P_DEF']
        for n in numeric_ind:
            temp[n] = float(temp[n])
        
        # add to dictionary
        data_dict[count] = temp
        count += 1
        
    return pd.DataFrame(data_dict).T

## Validation

In [13]:
def validate(df):
    """
    Runs checks on dataframe. Returns roster indices that need fixed.
    """
    
    # validate number of users
    unique_users = df['Username'].unique()
    users_correct = len(unique_users)
    
    print('Unique users:', users_correct)
    
    if users_correct != 100:
        # implement to find indices of multiple users
        pass
    
    # validate player names are in the right format
    name_format_wrong = []
    
    for i in df.index:
        if sum(df.loc[i, 'QB':'DEF'].isna()) == 9:
            continue
        # count entries where element at index i is a period
        if sum(df.loc[i, 'QB':'DEF'].str.find('.') == 1) != 9:
            name_format_wrong.append(i)
            
    print('Correct name formatting:', df.shape[0] - len(name_format_wrong))
    
    if len(name_format_wrong) > 0:
        # implement to find indices of incorrect name formatting
        pass
    
    # validate player scores equal roster score
    sum_of_players = df.loc[:, df.columns[df.columns.str.contains('P_')]].sum(axis='columns')
    score_correct = ((df['Score'] - sum_of_players).abs() < .01).sum()
    
    print('Score matches player scores:', score_correct)
    if score_correct != 100:
        # implement to find indices of incorrect scores
        pass
    
    return (users_correct == 100) and (score_correct == 100) and (len(name_format_wrong) == 0)


def save(df, gameid, contestid, update_log=False, log=None):
    """
    Save dataframe to disk and update Contest Reference log.
    """

    # save dataframe
    df.to_excel(f'saved contests/{gameid}-{contestid}.xlsx')
    
    # update and save log
    log.loc[log['URL'].str.contains(f'{gameid}-{contestid}'), 'Extracted'] = 'Auto extracted'
    
    log.to_excel('Contest Reference.xlsx', sheet_name='Scraping', index=False)

## Putting it all together

In [14]:
def extractData(skip=['Auto extracted']):
    # load URLs from reference
    url_data = getURLData()

    for line in url_data.index:
        url, extract = url_data.loc[line]
        gameid, contestid = parseIDs(url)

        # skip entries that have already been extracted
        if extract in skip:
            continue

        print('-----------------------------------------')
        print(f'Scraping data for {gameid}-{contestid}')

        # open URL and wait
        openURL(url)
        gui.click(200, 0, interval=3)
        
        # reposition
        repositionFanDuel()
        gui.click(200, 0, interval=.2)

        # locate keys on screen
        FIRST, NEXT, POSITION = locateKeys()

        # set patience interval
        PATIENCE = .25

        # while data is not valid, scrape and validate
        valid = False
        while not valid:
            # increment patience interval with each invalid attempt
            PATIENCE += .25
            
            # reset to first page
            gui.click(FIRST[0] + 5, FIRST[1] + 5, interval=PATIENCE*3)

            # scrape
            temp = scrape(pages=10, min_patience=PATIENCE, NEXT=NEXT, POSITION=POSITION)
            
            # clean
            df = cleanAllData(temp)

            # validate
            valid = validate(df)
            
            if valid:
                save(df, gameid, contestid, update_log=True, log=url_data)
                print('Saved successfully.')

            else:
                print('Did not pass validation!')
            print()
    

In [15]:
extractData()

-----------------------------------------
Scraping data for 85186-261917225
0:03:07.267410
Unique users: 99
Correct name formatting: 100
Score matches player scores: 100
Did not pass validation!

0:03:39.669046
Unique users: 100
Correct name formatting: 100
Score matches player scores: 100
Saved successfully.

-----------------------------------------
Scraping data for 85186-261916262


FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

In [16]:
url_data

NameError: name 'url_data' is not defined

In [19]:
df = pd.read_excel('saved contests/85186-261917225.xlsx', index_col=0)

In [20]:
df

,Experience,Username,Position,Winnings,Score,QB,RB1,RB2,WR1,WR2,...,DEF,P_QB,P_RB1,P_RB2,P_WR1,P_WR2,P_WR3,P_TE,P_FLEX,P_DEF
1,Highly experienced user,hendrickd73,1st of 100,3.6,167.22,M. Jones,J. McKinnon,T. Etienne Jr.,J. Meyers,M. Evans,...,P. Steelers,16.22,20.1,21.5,13.8,43.7,8.2,14.5,22.2,7
2,Experienced user,greeneb,2nd of 100,3.6,127.80,J. Goff,T. Etienne Jr.,L. Fournette,A. St. Brown,M. Evans,...,P. Steelers,22.40,21.5,6.7,8.2,43.7,0.4,3.4,14.5,7
3,Highly experienced user,bubbajay62,3rd of 100,3.6,126.82,P. Mahomes,J. McKinnon,M. Sanders,D. Moore,M. Brown,...,S. Seahawks,26.02,20.1,6.1,20.7,9.1,8.4,3.4,16.0,17
4,Highly experienced user,gnicholas,4th of 100,3.6,126.68,T. Brady,A. Kamara,T. Allgeier,T. Lockett,C. Godwin,...,N. Patriots,37.68,8.6,16.0,2.5,16.5,6.4,14.5,12.5,12
5,Experienced user,feldm,5th of 100,3.6,125.90,J. Goff,B. Robinson Jr.,T. Allgeier,A. St. Brown,R. James Jr.,...,N. Giants,22.40,8.7,16.0,8.2,17.1,16.0,3.4,20.1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Highly experienced user,kogrey,96th of 100,0.0,62.40,J. Goff,M. Sanders,B. Robinson Jr.,J. Jefferson,A. St. Brown,...,M. Vikings,22.40,6.1,8.7,2.0,8.2,3.5,3.4,6.1,2
97,Highly experienced user,hacantor,97th of 100,0.0,59.00,J. Fields,M. Sanders,R. Stevenson,C. Kirk,A. Lazard,...,L. Raiders,17.20,6.1,6.1,3.1,8.4,2.0,3.4,14.7,-2
98,Highly experienced user,ssrick,98th of 100,0.0,58.66,A. Rodgers,R. Stevenson,M. Sanders,T. Lockett,C. Samuel,...,L. Raiders,16.56,6.1,6.1,2.5,1.1,8.2,7.8,12.3,-2
99,Highly experienced user,my5cents,99th of 100,0.0,55.88,T. Lawrence,A. Kamara,N. Harris,J. Jefferson,A. St. Brown,...,L. Raiders,5.48,8.6,19.3,2.0,8.2,8.4,3.4,2.5,-2


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 1 to 100
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Experience  100 non-null    object 
 1   Username    100 non-null    object 
 2   Position    100 non-null    object 
 3   Winnings    100 non-null    float64
 4   Score       100 non-null    float64
 5   QB          100 non-null    object 
 6   RB1         100 non-null    object 
 7   RB2         100 non-null    object 
 8   WR1         100 non-null    object 
 9   WR2         100 non-null    object 
 10  WR3         100 non-null    object 
 11  TE          100 non-null    object 
 12  FLEX        100 non-null    object 
 13  DEF         100 non-null    object 
 14  P_QB        100 non-null    float64
 15  P_RB1       100 non-null    float64
 16  P_RB2       100 non-null    float64
 17  P_WR1       100 non-null    float64
 18  P_WR2       100 non-null    float64
 19  P_WR3       100 non-null    f